## LAB-16 Python 下載網頁

* 使用 requests.get() 取得網頁內容
* 使用 open() 開新檔籍儲存
* 使用 tqdm 顯示下載進度條

程式範例根據 Claude AI 改寫  

使用之提示詞
> Python 如何將網址下載儲存為檔案，並使用 tqdm 顯示下載進程


* [ChatGPT](https://chatgpt.com/share/67d3829a-253c-8009-93b5-3fb005521fdf)
* [Google Gemini](https://g.co/gemini/share/7910d428b59d)
* [Claude AI](https://claude.ai/share/aba9d3a0-aac0-4e0e-ade5-2befa1f0e3aa)

In [ ]:
import os                                        # 作業系統及檔案
import requests                                  # HTTP 連線
from tqdm.notebook import tqdm                   # 進度條 Progress bar
import time

In [ ]:
def download_file(url, filename=None):
    """
    從指定 URL 下載檔案，並使用 tqdm 顯示下載進度。
    
    參數:
    url (str): 要下載的檔案網址
    filename (str, optional): 要儲存的本地檔案名稱，如果未指定則從 URL 提取
    
    返回:
    str: 下載的檔案路徑
    """
    
    # 如果沒有指定檔名，則從 URL 中取得
    if filename is None:
        filename = os.path.basename(url)
    
    # 發送 GET 請求，設定 stream=True 以啟用串流下載
    response = requests.get(url, stream=True)
    
    # 確認請求成功
    response.raise_for_status()
    
    # 取得檔案大小（單位：位元組）
    total_size = int(response.headers.get('content-length', 0))

    # for debug
    read_bytes = 0

    bar = tqdm(desc=filename, total=total_size, unit='B', unit_scale=True, unit_divisor=1024)
    
    bar.refresh()
    
    # 準備寫入檔案
    with open(filename, 'wb') as file:
        
        # 逐塊下載資料
        for chunk in response.iter_content(chunk_size=1024):
            if chunk:  # 確保不是空值
                read_bytes += len(chunk)
                #print(f"{read_bytes}/{total_size} = {read_bytes/total_size:.2%}")

                bar.update(len(chunk))
                #bar.refresh()
                
                # 將下載的資料寫入檔案並更新進度條
                file.write(chunk)

                # for debug 
                #time.sleep(0.1)
    
    #bar.close()            
    #del bar 
    
    return filename


In [ ]:
# 監察院 API 網址
CY_URL = r"https://ardata.cy.gov.tw/api/v1/search/export/"
PAGE_SIZE = 1_000

def get_api_url(ecode:str, page_no:int) -> str:
    """
    回傳監察員政治線間查詢平台資料下載網址

    參數
    ecode        選舉代碼：總統大選 113101，立法委員 113102
    page_no      下載頁碼

    回傳值
    api_url 
    """

    api_url = f"{CY_URL}?page={page_no}&pageSize={PAGE_SIZE}&electionCode={ecode}"

    return api_url
    

In [ ]:
# 測試 API URL
page_no = 1
election = "113102"

url = get_api_url(election, page_no)
print(url)

In [ ]:
# 可人工透過 Browser 測試

In [ ]:
def create_data_path(path_name):
    """
    檢查目錄是否存在，如果不存在則創建該目錄
    
    參數:
    path_name (str): 要檢查或創建的目錄名稱
    
    返回:
    bool: 如果目錄已存在返回 False，如果目錄被創建返回 True
    """
    if not os.path.exists(path_name):
        os.makedirs(path_name)
        print(f"目錄 '{path_name}' 已新增")
        return
    else:
        print(f"目錄 '{path_name}' 已存在")
        return


In [ ]:
# 資料儲存目錄
data_path = "DATA"
create_data_path(data_path)

In [ ]:
# 資料總頁數
total_pages = 189                                # 參考之前網頁內容
csv_name = "2024立法委員2"                         # 儲存之檔名，不含頁碼及副檔名

total_pages = 2                                # 測試用，避免花太多時間

In [ ]:
# 下載所有檔案
for i in tqdm(range(1,total_pages+1), desc="資料下載"):
    url = get_api_url(election, i)
    fname = f"{csv_name}_{i}.csv"
    csvf = os.path.join(data_path, fname)
    csv_file = download_file(url, csvf)
    

In [ ]:
# TODO 下載進度條，並未如其顯示進度，而是 0-100% 
# TODO 第二層的 Progress Bar 會重複顯示


In [ ]:

!ls -ltr {data_path}

### TODO tqdm 
若有二層 tqdm 
1. 先 create 2 個 tqdm
2. 第一層使用 for in range() 而不是 for in tqdm(range)，於迴圈中更新進度
3. 將第二層 tqdm 傳入 download_file() 於下載前重設及每個 chunk 更新
